# ReAct Agent Demo — Reasoning + Tool Use + Evidence

Goal:
- Show how an agent thinks, calls tools, observes results, and forms an answer you can defend.
- Show the full trace so leadership can audit how the answer was formed.

Pattern:
1. Thought: what do I need?
2. Action: call tool
3. Observation: tool output
4. (repeat)
5. Final Answer

Scenario:
"Find an executive-suitable hotel near Hitech City in Hyderabad for Monday, and tell me what to pack based on current weather."

In [ ]:
import os, requests, textwrap
OPENWEATHER_API_KEY = os.environ.get('OPENWEATHER_API_KEY', '<PUT_YOUR_KEY_HERE>')
WIKI_HEADERS = {'User-Agent': 'react-agent-notebook/1.0'}

def wikipedia_summary(topic: str) -> str:
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    r = requests.get(url, headers=WIKI_HEADERS, timeout=10)
    if r.status_code != 200:
        return f"[wiki] error {r.status_code}: {r.text[:200]}"
    d = r.json()
    return f"{d.get('title','')} — {d.get('description','')}\n{d.get('extract','')}"

def weather_brief(latlon: str) -> str:
    if OPENWEATHER_API_KEY.startswith('<PUT_'):
        return '[weather] Please set OPENWEATHER_API_KEY'
    lat, lon = [float(x.strip()) for x in latlon.split(',')]
    url = 'https://api.openweathermap.org/data/2.5/weather'
    params = {'lat': lat, 'lon': lon, 'appid': OPENWEATHER_API_KEY}
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return f"[weather] error {r.status_code}: {r.text[:200]}"
    d = r.json()
    k2c = lambda k: round(k - 273.15, 1)
    desc = d['weather'][0]['description']
    temp_c = k2c(d['main']['temp'])
    return f"{desc}, {temp_c}°C"

def corporate_hotel(city: str) -> str:
    if city.lower() == 'hyderabad':
        return (
            'MetroLink Executive Suites (~₹5400/night). '
            '5-10 min to Hitech City. Breakfast, meeting room, gym. '
            'Commonly approved for VP/executive travel.'
        )
    return 'No pre-approved executive hotel found for that city.'


### Simulated ReAct Trace
Below, we simulate what a ReAct-style LLM would output. In production you'd drive this with an actual reasoning model (for example, `o4-mini`) and let it iteratively produce `Thought`, `Action`, and `Observation` steps before the `Final Answer`. Here we construct that trace explicitly so the notebook runs anywhere without needing an API call.


In [ ]:
def react_demo(user_query: str):
    trace = []

    step1 = textwrap.dedent('''
    Thought: The user is asking about an executive-suitable hotel near Hitech City in Hyderabad and weather/packing tips.
    Thought: I should confirm basic context about Hyderabad first.
    Action: wikipedia_summary
    Action Input: Hyderabad
    ''').strip()
    trace.append(step1)
    obs1 = wikipedia_summary('Hyderabad')
    trace.append('Observation: ' + obs1)

    step2 = textwrap.dedent('''
    Thought: I should get current weather so I can advise packing.
    Action: weather_brief
    Action Input: 17.44,78.38
    ''').strip()
    trace.append(step2)
    obs2 = weather_brief('17.44,78.38')
    trace.append('Observation: ' + obs2)

    step3 = textwrap.dedent('''
    Thought: I should retrieve a policy-approved executive hotel near Hitech City.
    Action: corporate_hotel
    Action Input: Hyderabad
    ''').strip()
    trace.append(step3)
    obs3 = corporate_hotel('Hyderabad')
    trace.append('Observation: ' + obs3)

    final_answer = textwrap.dedent(f'''
    Final Answer: Hyderabad is a major Indian tech/business hub around Hitech City.
    Recommended stay: {obs3}
    Weather right now: {obs2}
    Guidance: Stay within ~10 min of Hitech City to cut transit risk. Pack for actual conditions.
    Mention this approved property in your expense note to stay within reimbursement expectations.
    ''').strip()
    trace.append(final_answer)

    return '\n\n'.join(trace)

demo_query = (
    'Find me an exec-suitable hotel near Hitech City in Hyderabad for Monday. '
    'Include cost logic and packing/weather tips.'
)
print(react_demo(demo_query))


### Why leadership cares
- You can scroll up and literally *show the evidence chain*.
- You can defend why a recommendation was made: which data, which assumptions.
- This is how you convince security / compliance / finance that the AI isn't inventing policy.

ReAct is not just clever prompting. It's auditability.